In [ ]:
%pip install torch accelerate torchaudio datasets
%pip install --upgrade transformers

In [ ]:
%pip install git+https://github.com/huggingface/transformers.git


In [ ]:
import os
from huggingface_hub import HfApi, login

# Load the token from environment variable
hf_token = ""
login(token=hf_token)
# Initialize the HfApi with the token
api = HfApi(token=hf_token)

# Now you can use the API client to interact with Hugging Face Hub



In [ ]:
!huggingface-cli login

In [ ]:
%pip install soundfile
%pip install librosa

In [ ]:
from datasets import load_dataset, Audio

# English
stream_data = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="test", streaming=True)
stream_data = stream_data.cast_column("audio", Audio(sampling_rate=16000))
en_sample = next(iter(stream_data))["audio"]["array"]


In [ ]:
stream_data

In [ ]:
# French
stream_data = load_dataset("mozilla-foundation/common_voice_13_0", "fr", split="test", streaming=True)
stream_data = stream_data.cast_column("audio", Audio(sampling_rate=16000))
fr_sample = next(iter(stream_data))["audio"]["array"]

In [1]:
from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch

model_id = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)


Some weights of the model checkpoint at facebook/mms-1b-all were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/mms-1b-all and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

In [ ]:
en_sample

In [4]:
inputs = processor(en_sample, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
# 'joe keton disapproved of films and buster also had reservations about the media'


In [ ]:
transcription

In [ ]:
inputs

In [ ]:
en_sample

In [5]:
import librosa 
audio_path= "1102 b and example - Male Voice.wav"
sampling_rate=16000
audio, _ = librosa.load(audio_path, sr=sampling_rate)

# Check and adjust audio sampling rate if necessary
# if audio.sr != sampling_rate:
#     print('Check Failed')
#     audio = librosa.resample(audio, orig_sr=audio.sr, target_sr=sampling_rate)
    

In [ ]:
audio

In [ ]:
processor.tokenizer.set_target_lang("kmr-script_latin")
model.load_adapter("kmr-script_latin")

inputs = processor(audio, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
print (transcription)

In [ ]:
transcription

In [1]:
from transformers import Wav2Vec2ForCTC, AutoProcessor

model_id = "facebook/mms-1b-fl102"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)

Some weights of the model checkpoint at facebook/mms-1b-fl102 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/mms-1b-fl102 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-st

In [3]:
from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch
from pydub import AudioSegment
import os
import librosa 

model_id = "facebook/mms-1b-fl102"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)

def split_audio_to_chunks(file_path, output_dir, chunk_length_ms=60000):
    # Load the audio file
    audioFull = AudioSegment.from_file(file_path)
    
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Get the length of the audio file in milliseconds
    audio_length_ms = len(audioFull)
    
    final_transcription = ""
    
    # Split the audio file into chunks
    for i in range(0, audio_length_ms, chunk_length_ms):
        # Determine the start and end of the chunk
        start_ms = i
        end_ms = min(i + chunk_length_ms, audio_length_ms)
        
        # Extract the chunk
        chunk = audioFull[start_ms:end_ms]
        
        
        # Create the output file name
        chunk_filename = os.path.join(output_dir, f"chunk_{i // chunk_length_ms + 1}.wav")
        
        # Export the chunk to a file
        chunk.export(chunk_filename, format="wav")
        
        print(f"Exported {chunk_filename}")
        
        audio_path= chunk_filename
        sampling_rate=16000
        audio, _ = librosa.load(audio_path, sr=sampling_rate)
        
        processor.tokenizer.set_target_lang("gle")
        model.load_adapter("gle")

        inputs = processor(audio, sampling_rate=16_000, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**inputs).logits

        ids = torch.argmax(outputs, dim=-1)[0]
        transcription = processor.decode(ids)
        print (transcription)
        final_transcription = final_transcription + transcription

    return final_transcription

# Example usage
input_file = "President of Ireland speaking in Gaelic (St. Patrick s Day Message).mp3"  # Replace with your audio file path
output_directory = "tempAudio_chunks"  # Replace with your desired output directory
transcription = split_audio_to_chunks(input_file, output_directory)


Exported tempAudio_chunks\chunk_1.wav
beainachtaí na féila páidrigh ar chlean mhór ghónda na ngéil sa bhaille agus ar fud na cruinne ar ar lá náisiúnta ceilúra 1n tá rimead orm bainictaí na féila páidrigh a chur ar gach duine atá páirteag a geillirad na bliana seo is cumad ce acu a ndéirann nó tharsáile atá siad tugann ceilúra na féila páidrigh déis ar leith dúinn ar nasc éighreacht agus canna a threisiú le chlan gón da na ngéil agus in ar gairde ar fud an dón is léir a iontach é ar an luathair sin na beatha tagaidh na ngéal agus is mó an cuais atá agaoinn ceiliúra a dheannamh i mbliana ná mar a bhí le tamall de bhlianta chur abh fáilte libh an deabhlodí rialta ise atá dúiscart aerann agus a túsadh chur anis le réi nua ar oileann na héireann a thugann galántaí síochanna reatha agus comhphairtíocht a dúann uile ba bhaith lom baíochas a gal libh go léir as an tacaíocht agus an spreagú a thug sibh duinn
Exported tempAudio_chunks\chunk_2.wav
nar duras ad dheacar go dtí an pointe cothintait

In [ ]:
transcription